### Building insults detector using python

The code below is an alternative to writing a [config file](https://github.com/deepmipt/DeepPavlov/blob/0.17.1/deeppavlov/configs/classifiers/insults_kaggle_bert_torch.json) and using it with 

```python
model = build_model('insults_kaggle_bert_torch', download=True)
```

In [ ]:
from pathlib import Path

from deeppavlov import Element, Model
from deeppavlov.core.data.simple_vocab import SimpleVocabulary
from deeppavlov.download import download_resource
from deeppavlov.models.classifiers.proba2labels import Proba2Labels
from deeppavlov.models.preprocessors.torch_transformers_preprocessor import TorchTransformersPreprocessor
from deeppavlov.models.torch_bert.torch_transformers_classifier import TorchTransformersClassifierModel


model_path = Path('~/.deeppavlov/models/classifiers/insults_kaggle_torch_bert')

# Downloading pretrained model
download_resource(
    'http://files.deeppavlov.ai/deeppavlov_data/classifiers/insults_kaggle_torch_bert_v0.tar.gz',
    {'~/.deeppavlov/models/classifiers'}
)

preprocessor = TorchTransformersPreprocessor(
    vocab_file='bert-base-uncased',
    do_lower_case=True,
    max_seq_length=64
)

classes_vocab = SimpleVocabulary(
    save_path=model_path/'classes.dict',
    load_path=model_path/'classes.dict'
)

classifier = TorchTransformersClassifierModel(
    n_classes=classes_vocab.len,
    return_probas=True,
    pretrained_bert='bert-base-uncased',
    save_path=model_path/'model',
    load_path=model_path/'model',
    optimizer='AdamW',
    optimizer_parameters={'lr': 1e-05},
    learning_rate_drop_patience=5,
    learning_rate_drop_div=2.0
)

proba2labels = Proba2Labels(max_proba=True)

model = Model(
    x=['x'],
    out=['y_pred_labels'],
    pipe=[
        Element(component=preprocessor, x=['x'], out=['bert_features']),
        Element(component=classifier, x=['bert_features'], out=['y_pred_probas']),
        Element(component=proba2labels, x=['y_pred_probas'], out=['y_pred_ids']),
        Element(component=classes_vocab, x=['y_pred_ids'], out=['y_pred_labels'])
    ]
)

# Using model
model(['you are stupid', 'you are smart'])